## Subtask A: Entity and Relationship Extraction


In [3]:
#pip install transformers
#pip install spacy
#pip install nltk
#pip install torch

SyntaxError: invalid syntax (1011913231.py, line 5)

In [ ]:
!python -m spacy download en_core_web_sm

In [4]:
import sqlite3
from transformers import AutoTokenizer, AutoModelForTokenClassification, pipeline
import spacy 
import nltk
import requests 
import torch
import pandas as pd
from bs4 import BeautifulSoup
pd.set_option("display.max_rows", 200)

c:\Users\ongai\Desktop\DSA3101\WW-AI-GK\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [7]:
dbpath = 'data/ecmdatabase.db'
con = sqlite3.connect(f"file:{dbpath}?mode=ro", uri=True)
with con:
    result = con.execute("SELECT item1 from companies WHERE stock_symbol = 'TSLA';")
    tsla_item1 = result.fetchall()[0][0]

In [8]:
tsla_item1

" ITEM 1. BUSINESS \n\nOverview \n\nWe design, develop, manufacture, sell and lease high-performance fully electric vehicles and energy generation and storage systems, and offer services related to our products. We generally sell our products directly to customers, and continue to grow our customer-facing infrastructure through a global network of vehicle showrooms and service centers, Mobile Service, body shops, Supercharger stations and Destination Chargers to accelerate the widespread adoption of our products. We emphasize performance, attractive styling and the safety of our users and workforce in the design and manufacture of our products and are continuing to develop full self-driving technology for improved safety. We also strive to lower the cost of ownership for our customers through continuous efforts to reduce manufacturing costs and by offering financial and other services tailored to our products. \n\nOur mission is to accelerate the world&#8217;s transition to sustainable

In [26]:
tsla_item1 = tsla_item1.replace('\n', '')
tsla_item1

" ITEM 1. BUSINESS Overview We design, develop, manufacture, sell and lease high-performance fully electric vehicles and energy generation and storage systems, and offer services related to our products. We generally sell our products directly to customers, and continue to grow our customer-facing infrastructure through a global network of vehicle showrooms and service centers, Mobile Service, body shops, Supercharger stations and Destination Chargers to accelerate the widespread adoption of our products. We emphasize performance, attractive styling and the safety of our users and workforce in the design and manufacture of our products and are continuing to develop full self-driving technology for improved safety. We also strive to lower the cost of ownership for our customers through continuous efforts to reduce manufacturing costs and by offering financial and other services tailored to our products. Our mission is to accelerate the world&#8217;s transition to sustainable energy. We 

## Data Exploration - Text Analysis

### Frequency Analysis

In [50]:
## FREQUENCY ANALYSIS
from collections import Counter
# load spacy model
nlp = spacy.load('en_core_web_sm')
nlp.disable_pipe("parser")
nlp.enable_pipe("senter")
# load data
doc = nlp(tsla_item1)
words = [token.text for token in doc if not token.is_stop and not token.is_punct]

print(Counter(words).most_common(20))

[('energy', 75), ('vehicles', 68), ('systems', 39), ('vehicle', 34), ('products', 33), ('solar', 33), ('storage', 32), ('customers', 32), ('Tesla', 32), ('U.S.', 25), ('electric', 24), ('including', 24), ('certain', 23), ('battery', 20), ('driving', 18), ('new', 17), ('offer', 16), ('self', 16), ('markets', 16), ('Energy', 16)]


## Named Entity Recognition

### NLTK

In [17]:
## using nltk
nltk.download('punkt')
nltk.download('punkt_tab')
nltk.download('averaged_perceptron_tagger_eng')
nltk.download('maxent_ne_chunker_tab')
nltk.download('words')
 
# Step Two: Load Data
print(len(tsla_item1))

# Step Three: Tokenise, find parts of speech and chunk words 

for sent in nltk.sent_tokenize(tsla_item1):
  for chunk in nltk.ne_chunk(nltk.pos_tag(nltk.word_tokenize(sent))):
     if hasattr(chunk, 'label'):
        print(chunk.label(), ' '.join(c[0] for c in chunk))

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\ongai\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt_tab to
[nltk_data]     C:\Users\ongai\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger_eng to
[nltk_data]     C:\Users\ongai\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger_eng is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package maxent_ne_chunker_tab to
[nltk_data]     C:\Users\ongai\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping chunkers\maxent_ne_chunker_tab.zip.
[nltk_data] Downloading package words to
[nltk_data]     C:\Users\ongai\AppData\Roaming\nltk_data...
[nltk_data]   Package words is already up-to-date!


46161
ORGANIZATION BUSINESS
PERSON Mobile Service
GPE Supercharger
ORGANIZATION Services Automotive
GPE Y
GPE S
ORGANIZATION Cybertruck
PERSON Model
ORGANIZATION SUV
PERSON Model
PERSON Model X
ORGANIZATION SUV
PERSON Model
PERSON Model X
ORGANIZATION SUV
ORGANIZATION Cybertruck
ORGANIZATION Tesla Semi
ORGANIZATION FSD
PERSON Energy
ORGANIZATION Generation
PERSON Storage Energy Storage
GPE Powerwall
GPE Megapack
ORGANIZATION GWh
PERSON Solar
PERSON Energy Offerings
ORGANIZATION PPA
PERSON Technology
ORGANIZATION Automotive Battery
GPE Powertrain
PERSON Model S
PERSON Model X
PERSON Cybertruck
ORGANIZATION Tesla Semi
PERSON Vehicle
ORGANIZATION Control
ORGANIZATION Infotainment
GSP Control
ORGANIZATION Artificial
ORGANIZATION FSD Computer
ORGANIZATION FSD
GPE Optimus
PERSON Energy
ORGANIZATION Generation
PERSON Storage Energy Storage
PERSON Solar
ORGANIZATION Energy Systems
PERSON Solar Roof
ORGANIZATION Powerwall
GPE Design
PERSON Energy
ORGANIZATION Generation
PERSON Solar Roof
PERSON

### Spacy

In [40]:
## Using spacy

# load spacy model
nlp = spacy.load('en_core_web_sm')
nlp.disable_pipe("parser")
nlp.enable_pipe("senter")
# load data
doc = nlp(tsla_item1)

# collect unique labels
unique_labels = {}
for ent in doc.ents:
    if ent.label_ not in unique_labels.keys():
        unique_labels[ent.label_] = []
    unique_labels[ent.label_].append((ent.text, ent.start_char, ent.end_char))


- PERSON:      People, including fictional.
- NORP:        Nationalities or religious or political groups.
- FAC:         Buildings, airports, highways, bridges, etc.
- ORG:         Companies, agencies, institutions, etc.
- GPE:         Countries, cities, states.
- LOC:         Non-GPE locations, mountain ranges, bodies of water.
- PRODUCT:     Objects, vehicles, foods, etc. (Not services.)
- EVENT:       Named hurricanes, battles, wars, sports events, etc.
- WORK_OF_ART: Titles of books, songs, etc.
- LAW:         Named documents made into laws.
- LANGUAGE:    Any named language.
- DATE:        Absolute or relative dates or periods.
- TIME:        Times smaller than a day.
- PERCENT:     Percentage, including ”%“.
- MONEY:       Monetary values, including unit.
- QUANTITY:    Measurements, as of weight or distance.
- ORDINAL:     “first”, “second”, etc.
- CARDINAL:    Numerals that do not fall under another type.

In [41]:
unique_labels.keys()

dict_keys(['ORG', 'PERSON', 'CARDINAL', 'MONEY', 'PRODUCT', 'NORP', 'DATE', 'ORDINAL', 'TIME', 'LOC', 'GPE', 'FAC', 'EVENT', 'LAW', 'PERCENT', 'WORK_OF_ART'])

In [42]:
unique_labels['ORG']

[('Mobile Service', 380, 394),
 ('Our Products and Services Automotive', 1942, 1978),
 ('Full Self-Driving (&', 3066, 3086),
 ('Energy Generation and Storage Energy Storage Products Powerwall',
  3162,
  3225),
 ('Solar Energy Offerings', 3925, 3947),
 ('Powertrain Our', 4680, 4694),
 ('Model S', 5321, 5328),
 ('Vehicle Control', 5785, 5800),
 ('Control systems', 5942, 5957),
 ('Self-Driving Development and Artificial Intelligence We', 6250, 6305),
 ('FSD Computer', 6449, 6461),
 ('Autopilot', 6804, 6813),
 ('Tesla', 7312, 7317),
 ('AI', 7652, 7654),
 ('Energy Generation and Storage Energy Storage Products', 7663, 7716),
 ('Solar Energy Systems', 8235, 8255),
 ('Energy Generation and Storage', 9642, 9671),
 ('Megapack', 9945, 9953),
 ('Automotive Direct Sales Our', 10536, 10563),
 ('Tesla', 11052, 11057),
 ('Tesla', 11508, 11513),
 ('Tesla', 11585, 11590),
 ('Tesla', 11737, 11742),
 ('Tesla Superchargers', 11857, 11876),
 ('the North American Charging Standard', 12677, 12713),
 ('NACS'

In [44]:
unique_labels['PRODUCT'] #products

[('the Model 3', 2045, 2056),
 ('Model 3', 2082, 2089),
 ('Model 3', 2277, 2284),
 ('Model S', 2437, 2444),
 ('Solar Roof', 4355, 4365),
 ('Solar Roof', 8275, 8285),
 ('Solar Roof', 10158, 10168),
 ('Solar Roof', 16095, 16105),
 ('Solar Roof', 17592, 17602),
 ('Model 3', 32041, 32048),
 ('Model Y', 32053, 32060)]

In [45]:
unique_labels['LOC'] # locations

[('North America', 16709, 16722),
 ('Europe', 16724, 16730),
 ('Asia', 16735, 16739),
 ('Northern California', 17880, 17899),
 ('Europe', 32867, 32873),
 ('Europe', 41224, 41230)]

In [46]:
unique_labels['FAC'] # facilities / factories

[('Gigafactory Texas', 17963, 17980),
 ('Fremont Factory', 42324, 42339),
 ('Gigafactory Texas', 42361, 42378)]

In [47]:
unique_labels['EVENT'] # events

[('this Annual Report on Form 10-K', 20814, 20845),
 ('this Annual Report on Form 10-K. Energy Storage System Incentives',
  21988,
  22053),
 ('this Annual Report on Form 10-K. Pursuant', 23163, 23204),
 ('this Annual Report on Form 10-K.', 45826, 45858)]

In [49]:
from spacy import displacy
#displacy.render(doc, style="ent")

### Hugging Face Transformers

Because entities can be of different types and not all are equally important in the
context of the natural language text being analyzed, it is quite common for NER
processors to return the following in addition to a list of entities:

 **type**
- Is it a person? Is it a location? Is it an organization? The set of categories will
depend on the specific model used. 
- The bert-base-NER distinguishes four types
of entities: location (LOC), organization (ORG), person (PER), and miscellaneous
(MISC).
 
**salience**
- The relative importance in the text analyzed or, in other words, the entity’s
relevance. 
- Is the entity central to the text (higher score/salience), or is it just
mentioned tangentially (lower score/salience)?

In [9]:
## Using transformers
tokenizer = AutoTokenizer.from_pretrained("dslim/bert-base-NER")
model = AutoModelForTokenClassification.from_pretrained("dslim/bert-base-NER")
ner_pipe = pipeline("ner", model = model, tokenizer = tokenizer)
for ent in ner_pipe(tsla_item1):
    print(ent)

Some weights of the model checkpoint at dslim/bert-base-NER were not used when initializing BertForTokenClassification: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


{'entity': 'I-ORG', 'score': 0.54775363, 'index': 74, 'word': 'Service', 'start': 391, 'end': 398}
{'entity': 'B-MISC', 'score': 0.5205288, 'index': 84, 'word': 'Des', 'start': 438, 'end': 441}
{'entity': 'I-MISC', 'score': 0.62265116, 'index': 87, 'word': 'Chargers', 'start': 450, 'end': 458}
{'entity': 'B-ORG', 'score': 0.75424445, 'index': 350, 'word': 'Products', 'start': 1960, 'end': 1968}
{'entity': 'I-ORG', 'score': 0.9869141, 'index': 351, 'word': 'and', 'start': 1969, 'end': 1972}
{'entity': 'I-ORG', 'score': 0.9261165, 'index': 352, 'word': 'Services', 'start': 1973, 'end': 1981}
{'entity': 'I-ORG', 'score': 0.88983554, 'index': 353, 'word': 'Auto', 'start': 1984, 'end': 1988}
{'entity': 'I-ORG', 'score': 0.60575384, 'index': 354, 'word': '##mot', 'start': 1988, 'end': 1991}
{'entity': 'B-MISC', 'score': 0.59626323, 'index': 369, 'word': 'Model', 'start': 2067, 'end': 2072}
{'entity': 'I-MISC', 'score': 0.9495684, 'index': 370, 'word': '3', 'start': 2073, 'end': 2074}


## Diffbot API
https://www.diffbot.com/


In [65]:
from getpass import getpass

TOKEN = getpass('Enter token: ')

In [87]:
FIELDS = "entities,facts"
HOST = "nl.diffbot.com"

In [67]:
import json

def get_request(payload):
  res = requests.post("https://{}/v1/?fields={}&token={}".format(HOST, FIELDS, TOKEN), json=payload)
  ret = None
  try:
    ret = res.json()
  except:
    print("Bad response: " + res.text)
    print(res.status_code)
    print(res.headers)
  return ret

In [88]:
res = get_request({
    "content": tsla_item1,
    "lang": "en",
    "format": "plain text with title",
})

print (res)

{'entities': [{'name': 'Tesla Semi', 'diffbotUri': 'https://diffbot.com/entity/EF4g4ohJUPUeLKh-N-rvUyA', 'confidence': 0.9569246, 'salience': 0.8978014, 'isCustom': False, 'allUris': ['http://www.wikidata.org/entity/Q40008974'], 'allTypes': [{'name': 'skill', 'diffbotUri': 'https://diffbot.com/entity/EvfbHngnSNVOh7ZBM5XTywQ'}, {'name': 'product', 'diffbotUri': 'https://diffbot.com/entity/EgSPUye7QPcyQoPylO8biMQ'}, {'name': 'tool', 'diffbotUri': 'https://diffbot.com/entity/EKvoYuTx4P9WT35YUouc0ug'}, {'name': 'vehicle', 'diffbotUri': 'https://diffbot.com/entity/E4vBDPVu3OTq90vd1GhX5mw', 'dbpediaUri': 'http://dbpedia.org/ontology/MeanOfTransportation'}], 'mentions': [{'text': 'Tesla', 'beginOffset': 2922, 'endOffset': 2927, 'confidence': 0.9569246}, {'text': 'Tesla', 'beginOffset': 5361, 'endOffset': 5366, 'confidence': 0.9569246}]}, {'name': 'automotive battery', 'diffbotUri': 'https://diffbot.com/entity/EF6RvyvRnNsW6oAQQjtPb7g', 'confidence': 0.97791916, 'salience': 0.77393895, 'isCusto

### View Entities and Entity Types

In [93]:
def extract_entities(res):
    for ent in res["entities"]:
        if ent["salience"] > 0.5:
            print("Entity Name: " + ent['name'])
            print("Salience: " + str(ent['salience']))
            print("Entity Types:")
            print([ent_type["name"] for ent_type in ent['allTypes']])
            print()

In [89]:
for ent in res["entities"]:
    if ent["salience"] > 0.5:
        print("Entity Name: " + ent['name'])
        print("Salience: " + str(ent['salience']))
        print("Entity Types:")
        print([ent_type["name"] for ent_type in ent['allTypes']])
        print()

Entity Name: Tesla Semi
Salience: 0.8978014
Entity Types:
['skill', 'product', 'tool', 'vehicle']

Entity Name: automotive battery
Salience: 0.77393895
Entity Types:
['skill', 'product', 'tool']

Entity Name: Tesla Model X
Salience: 0.6964315
Entity Types:
['skill', 'product', 'tool', 'vehicle']

Entity Name: solar energy
Salience: 0.6094218
Entity Types:
[]

Entity Name: Tesla Autopilot
Salience: 0.5553389
Entity Types:
['skill', 'field of work', 'technology']

Entity Name: artificial intelligence
Salience: 0.5336387
Entity Types:
['skill', 'field of work', 'technology']

Entity Name: electric vehicle
Salience: 0.5048789
Entity Types:
['skill', 'product', 'tool', 'vehicle']



In [90]:
if "facts" in res:
    df = pd.DataFrame.from_dict(res["facts"])
    pd.options.display.max_columns = None
    pd.set_option('display.width', 1000)
    print(df.head())

Empty DataFrame
Columns: []
Index: []


### Supplementary Source: Wikipedia Article

In [91]:
tesla_wiki = "Tesla, Inc. (/ˈtɛslə/ TESS-lə or /ˈtɛzlə/ TEZ-lə[a]) is an American multinational automotive and clean energy company. Headquartered in Austin, Texas, it designs, manufactures and sells battery electric vehicles (BEVs), stationary battery energy storage devices from home to grid-scale, solar panels and solar shingles, and related products and services. \
    Tesla was incorporated in July 2003 by Martin Eberhard and Marc Tarpenning as Tesla Motors. Its name is a tribute to inventor and electrical engineer Nikola Tesla. In February 2004, Elon Musk joined as Tesla's largest shareholder; in 2008, he was named chief executive officer. In 2008, the company began production of its first car model, the Roadster sports car, followed by the Model S sedan in 2012, the Model X SUV in 2015, the Model 3 sedan in 2017, the Model Y crossover in 2020, the Tesla Semi truck in 2022 and the Cybertruck pickup truck in 2023. The Model 3 is the all-time best-selling plug-in electric car worldwide, and in June 2021 became the first electric car to sell 1 million units globally.[6] In 2023, the Model Y was the best-selling vehicle, of any kind, globally.[7][8][3] \
        Tesla is one of the world's most valuable companies in terms of market capitalization. In October 2021, Tesla temporarily became a trillion-dollar company, the seventh U.S. company to do so. In 2023, the company led the battery electric vehicle market, with 19.9% share. Also in 2023, the company was ranked 69th in the Forbes Global 2000.[9] As of March 2024, it is the world's most valuable automaker. Tesla has been the subject of lawsuits, government scrutiny, and journalistic criticism, stemming from allegations of multiple cases of whistleblower retaliation, worker rights violations such as sexual harassment and anti-union activities, safety defects leadings to dozens of recalls, the lack of a public relations department, and controversial statements from Musk including overpromising on the company's driving assist technology and product release timelines."

In [92]:
res = get_request({
    "content": tesla_wiki,
    "lang": "en",
    "format": "plain text with title",
})

In [94]:
extract_entities(res)

Entity Name: Marc Tarpenning
Salience: 0.9887775
Entity Types:
['person']

Entity Name: Martin Eberhard
Salience: 0.9880303
Entity Types:
['person']

Entity Name: Tesla
Salience: 0.97538745
Entity Types:
['organization']

Entity Name: battery electric vehicle
Salience: 0.8341532
Entity Types:
['skill', 'product', 'tool', 'vehicle']

Entity Name: Elon Musk
Salience: 0.75881004
Entity Types:
['person']

Entity Name: plug-in electric vehicle
Salience: 0.7428088
Entity Types:
['skill', 'product', 'tool', 'vehicle']

Entity Name: Tesla Model Y
Salience: 0.7421379
Entity Types:
['skill', 'product', 'tool', 'vehicle']

Entity Name: pickup truck
Salience: 0.741191
Entity Types:
['skill', 'product', 'tool', 'vehicle']

Entity Name: Tesla Model X
Salience: 0.7204369
Entity Types:
['skill', 'product', 'tool', 'vehicle']

Entity Name: market capitalisation
Salience: 0.71955776
Entity Types:
[]

Entity Name: multinational corporation
Salience: 0.7153199
Entity Types:
[]

Entity Name: Tesla Model S


In [96]:
if "facts" in res:
    df = pd.DataFrame.from_dict(res["facts"])
    pd.options.display.max_columns = None
    pd.set_option('display.width', 3000)
    print(df.head())

                                       humanReadable                                             entity                                           property                                              value  confidence                                           evidence                                         qualifiers
0   [Tesla] product type [battery electric vehicles]  {'name': 'Tesla', 'diffbotUri': 'https://diffb...  {'name': 'product type', 'diffbotUri': 'https:...  {'name': 'battery electric vehicles', 'confide...    0.949955  [{'passage': 'Headquartered in Austin, Texas, ...                                                NaN
1  [Elon Musk] position held [chief executive off...  {'name': 'Elon Musk', 'diffbotUri': 'https://d...  {'name': 'position held', 'diffbotUri': 'https...  {'name': 'chief executive officer', 'confidenc...    0.937818  [{'passage': 'In February 2004, Elon Musk join...  [{'property': {'name': 'is not current'}, 'val...
2        [Tesla] chief executive officer